In [ ]:
!cat requirements.txt

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git

In [ ]:
PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
resource_id = "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file = "token.txt"

In [ ]:
with open(token_file, "r") as f:
    my_token = f.read()

In [ ]:
import json
from pprint import pprint

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats

import PicSureHpdsLib
import PicSureClient

from python_lib.utils import get_multiIndex_variablesDict, joining_variablesDict_onCol

In [ ]:
client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, my_token, True)
adapter = PicSureHpdsLib.Adapter(connection)
resource = adapter.useResource(resource_id)

In [ ]:
dictionary = resource.dictionary()
dictionary_search = dictionary.find("\\_consents\\").DataFrame()
consentGroups = dictionary_search["categoryValues"]["\\_consents\\"]
consentGroups

In [ ]:
import time

my_query = resource.query()
my_query.filter().delete("\\_topmed_consents\\")
my_query.filter().delete("\\_harmonized_consent\\")
dictionary = resource.dictionary()
output = []
batchSize = 1000
badConcepts = []
for consentGroup in consentGroups:
    if ".c0" not in consentGroup:
        print(consentGroup)
        consentGroupConcepts = dictionary.find(consentGroup.split(".")[0]).DataFrame()
        conceptsWithCounts = 0
        for i in range(0, len(consentGroupConcepts.index.values), batchSize):  
            my_query.crosscounts().clear()
            my_query.crosscounts().add(consentGroupConcepts.index.values[i:i + batchSize])
            my_query.filter().delete("\\_consents\\")
            my_query.filter().add("\\_consents\\", consentGroup)
            try:                
                cross_counts = my_query.getCrossCounts()
                cross_counts_list = [(k, v) for k, v in cross_counts.items() if v > 0] 
                cross_counts_empty = [(k, v) for k, v in cross_counts.items() if v == 0] 
                print(i)
                print(len(cross_counts_list))
                conceptsWithCounts += len(cross_counts_list)
            except:
                print("Retrying")
                time.sleep(5)
                badConcepts.extend(consentGroupConcepts.index.values[i:i + batchSize])
                cross_counts = my_query.getCrossCounts()
                cross_counts_list = [(k, v) for k, v in cross_counts.items() if v > 0] 
                cross_counts_empty = [(k, v) for k, v in cross_counts.items() if v == 0] 
                print("RETRY SUCCESS")
                print(i)
                print(len(cross_counts_list))
                conceptsWithCounts += len(cross_counts_list)
                pass
        output.append(consentGroup + "," + str(conceptsWithCounts))

In [ ]:
for line in output:
    print(line)

In [ ]:
my_query = resource.query()
my_query.filter().delete("\\_topmed_consents\\")
my_query.filter().delete("\\_harmonized_consent\\")
dictionary = resource.dictionary()
output = []
batchSize = 100
badConcepts = []
for consentGroup in consentGroups:
    if ".c0" not in consentGroup:
        print(consentGroup)
        my_query.filter().delete("\\_consents\\")
        my_query.filter().add("\\_consents\\", consentGroup)
        patient_count = my_query.getCount()
        print(patient_count)
        output.append(consentGroup + "," + str(patient_count))

In [ ]:
for line in output:
    print(line)